In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional, TimeDistributed, BatchNormalization
from dataset import LABELS
from sklearn.preprocessing import LabelEncoder
from dataset import label_detection_training, label_detection_validation, silence_detection_training, silence_detection_validation
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical
from dataset import TensorflowDataset

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# train = pd.read_pickle('extracted_features\\features_training.pkl')
# val = pd.read_pickle('extracted_features\\features_validation.pkl')

# X_train = np.array([x[0] for x in train])
# y_train = np.array([x[1] for x in train])
# X_val = np.array([x[0] for x in val])
# y_val = np.array([x[1] for x in val])

# # Create a label encoder object
# label_encoder = LabelEncoder()

# # Fit the label encoder using your labels (combine both train and val labels if they have different classes)
# label_encoder.fit(np.concatenate((y_train, y_val)))

# # Transform your string labels to integer labels for both training and validation sets
# y_train_encoded = label_encoder.transform(y_train)
# y_val_encoded = label_encoder.transform(y_val)

In [3]:
train = pd.read_pickle('extracted_features\\features_training.pkl')
y_train = np.array([x[1] for x in train])
labels = list(np.unique(y_train))
train = TensorflowDataset('extracted_features\\features_training.pkl', labels=labels).dataset
train = train.shuffle(len(train), reshuffle_each_iteration=True)
val = TensorflowDataset('extracted_features\\features_validation.pkl', labels=labels).dataset
val = val.shuffle(len(val), reshuffle_each_iteration=True)

In [5]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (MultiHeadAttention, Dropout, LayerNormalization, Dense, TimeDistributed,
                                     BatchNormalization, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D)
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

results = pd.DataFrame(columns=['num_heads', 'num_layers', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, dropout_rate):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential([
            Dense(embed_dim, activation='relu'),
            Dense(embed_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

for num_heads in [2, 4]:
    for num_layers in [1, 2]:
        for dropout_rate in [0.2, 0.4]:
            for batch_size in [32, 64]:
                for epochs in [10, 20]:
                    num_classes = len(np.unique(labels))  # Number of unique classes in your dataset
                    input_shape = (39, 44)
                    embed_dim = 128

                    inputs = tf.keras.Input(shape=input_shape)
                    x = Conv1D(filters=128, kernel_size=3, activation='relu')(inputs)
                    x = BatchNormalization()(x)
                    x = MaxPooling1D(pool_size=2)(x)
                    x = Dropout(dropout_rate)(x)

                    for _ in range(num_layers):
                        x = TransformerBlock(embed_dim, num_heads, dropout_rate)(x)

                    x = GlobalMaxPooling1D()(x)
                    x = Dense(512, activation='relu')(x)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)
                    x = Dense(256, activation='relu')(x)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)
                    x = Dense(128, activation='relu')(x)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)
                    outputs = Dense(num_classes, activation='softmax')(x)

                    model = tf.keras.Model(inputs=inputs, outputs=outputs)

                    # Compile the model
                    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                    # Train the model with your training set and validate it with your validation set
                    history = model.fit(train.batch(batch_size), epochs=epochs, batch_size=batch_size, validation_data=val.batch(batch_size), verbose=2)
                                    # Record the results for the current configuration
                    row = {
                        'num_heads': num_heads,
                        'num_layers': num_layers,
                        'dropout_rate': dropout_rate,
                        'epoch': epochs,
                        'batch': batch_size,
                        'loss': history.history['loss'][-1],
                        'loss_max': np.max(history.history['loss']),
                        'accuracy': history.history['accuracy'][-1],
                        'accuracy_max': np.max(history.history['accuracy']),
                        'val_loss': history.history['val_loss'][-1],
                        'val_loss_max': np.max(history.history['val_loss']),
                        'val_accuracy': history.history['val_accuracy'][-1],
                        'val_accuracy_max': np.max(history.history['val_accuracy']),
                    }

                    results = results.append(row, ignore_index=True)


Epoch 1/10
1811/1811 - 51s - loss: 2.1170 - accuracy: 0.4033 - val_loss: 1.3594 - val_accuracy: 0.5927 - 51s/epoch - 28ms/step
Epoch 2/10
1811/1811 - 53s - loss: 1.3355 - accuracy: 0.6051 - val_loss: 0.9565 - val_accuracy: 0.7123 - 53s/epoch - 29ms/step
Epoch 3/10
1811/1811 - 56s - loss: 1.1821 - accuracy: 0.6485 - val_loss: 0.8913 - val_accuracy: 0.7243 - 56s/epoch - 31ms/step
Epoch 4/10


In [ ]:
results.to_pickle('results\\model_transformer')